In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
import math

import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, TimeDistributed, BatchNormalization
from keras.callbacks import EarlyStopping

import datetime as dt

Using TensorFlow backend.


In [2]:
path = "C:/Users/OPTIMUSPRIME/Desktop/Studia/Magisterka/Faints-Prediction/"
BP_filename = "BP.csv"
HR_filename = "HR.csv"

train_indices =  [str(indx[0]) for indx in pd.read_csv(path + "DATA/training_set.txt").values.tolist()]
test_indices =  [str(indx[0]) for indx in pd.read_csv(path + "DATA/test_set.txt").values.tolist()]
validation_indices =  [str(indx[0]) for indx in pd.read_csv(path + "DATA/validation_set.txt").values.tolist()]
all_indices = train_indices + test_indices + validation_indices

In [3]:
BP_data = pd.read_csv(path + "DATA/" + BP_filename, low_memory=False)[all_indices]
HR_data = pd.read_csv(path + "DATA/" + HR_filename, low_memory=False)[all_indices]
labels = {col: BP_data[col].iloc[0] for col in BP_data}

BP_max_value = BP_data.iloc[1:].astype(np.float32).max().max()
BP_min_value = BP_data.iloc[1:].astype(np.float32).min().min()
HR_max_value = HR_data.iloc[1:].astype(np.float32).max().max()
HR_min_value = HR_data.iloc[1:].astype(np.float32).min().min()

BP_scaler = MinMaxScaler().fit(np.array([BP_min_value, BP_max_value]).reshape(-1,1))
HR_scaler = MinMaxScaler().fit(np.array([HR_min_value, HR_max_value]).reshape(-1,1))

BP_data_scaled = BP_data.iloc[1:].astype(np.float32).copy()
BP_data_scaled[all_indices] = BP_scaler.transform(BP_data_scaled[all_indices])

HR_data_scaled = HR_data.iloc[1:].astype(np.float32).copy()
HR_data_scaled[all_indices] = HR_scaler.transform(HR_data_scaled[all_indices])

def downsample_to(n, df, mean_window):
    new_df = pd.DataFrame()
    for col in df:
        data = df[col][1:]
#         data_temp = data.count()
        div = int(data.count() / n) - 1 # -1 bo jakies problemy sa ze avr_data po usrednieniu ma dlugosc < 200 i sa ujemne excessPoints
        avr_data = data.rolling(mean_window).mean()[::div]
        avr_data = [_ for _ in list(avr_data) if np.isnan(_) != True]
#         temp = len(avr_data)
        excess_points = len(avr_data) - n
        avr_data = avr_data[excess_points:]
        new_df[col] = pd.Series(avr_data)
#         print(f"TS {col} processing. Div: {div}. Excess points: {excess_points}. Dl: {temp}. data len: {data_temp}")
    return new_df

BP_data_scaled = downsample_to(200, BP_data_scaled, 50)
HR_data_scaled = downsample_to(200, HR_data_scaled, 50)

In [4]:
def split(BP_data, HR_data):
    X = []
    y = []
    for col in BP_data:
        BP_HR_interval = []
        for BP_interval, HR_interval in zip(BP_data[col], HR_data[col]):
            BP_HR_interval.append([BP_interval, HR_interval])
        X.append(BP_HR_interval)
        if labels[col] == 'Synkope': label = [0., 1.] 
        else: label = [1., 0.]
        y.append(label)
    return np.array(X), np.array(y)

In [5]:
X_train, y_train = split(BP_data_scaled[train_indices], HR_data_scaled[train_indices])
X_test, y_test = split(BP_data_scaled[test_indices], HR_data_scaled[test_indices])
X_validation, y_validation = split(BP_data_scaled[validation_indices], HR_data_scaled[validation_indices])

In [6]:
y_train.shape

(153, 2)

In [7]:
model = Sequential()
c=64
n=50
model.add(LSTM(c, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(c, return_sequences=True))
model.add(LSTM(c, return_sequences=True))
model.add(LSTM(c))
model.add(Dense(n, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


W0422 00:50:21.155788 20628 deprecation_wrapper.py:119] From C:\Programs\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0422 00:50:21.196244 20628 deprecation_wrapper.py:119] From C:\Programs\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0422 00:50:21.205222 20628 deprecation_wrapper.py:119] From C:\Programs\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0422 00:50:22.411896 20628 deprecation_wrapper.py:119] From C:\Programs\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0422 00:50:22.445807 20628 deprecation_wrapper.py:119] From C:\Programs\Anaconda3\lib\site-

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 200, 64)           17152     
_________________________________________________________________
lstm_2 (LSTM)                (None, 200, 64)           33024     
_________________________________________________________________
lstm_3 (LSTM)                (None, 200, 64)           33024     
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                3250      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 102       
Total params: 119,576
Trainable params: 119,576
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=128, verbose=1)
model.predict(X_validation)

W0422 00:50:22.675969 20628 deprecation.py:323] From C:\Programs\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0422 00:50:25.555685 20628 deprecation_wrapper.py:119] From C:\Programs\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 153 samples, validate on 19 samples
Epoch 1/5
153/153 [==============================] - 4s 29ms/step - loss: 0.6928 - acc: 0.5556 - val_loss: 0.6955 - val_acc: 0.4737
Epoch 2/5
153/153 [==============================] - 1s 9ms/step - loss: 0.6902 - acc: 0.5098 - val_loss: 0.6975 - val_acc: 0.4737
Epoch 3/5
153/153 [==============================] - 1s 10ms/step - loss: 0.6868 - acc: 0.5098 - val_loss: 0.7046 - val_acc: 0.4737
Epoch 4/5
153/153 [==============================] - 2s 10ms/step - loss: 0.6826 - acc: 0.5098 - val_loss: 0.7138 - val_acc: 0.4737
Epoch 5/5
153/153 [==============================] - 2s 10ms/step - loss: 0.6753 - acc: 0.5229 - val_loss: 0.7166 - val_acc: 0.4737


array([[0.5465911 , 0.4534089 ],
       [0.5048299 , 0.4951701 ],
       [0.55375636, 0.44624364],
       [0.49757397, 0.502426  ],
       [0.5271346 , 0.47286537],
       [0.55761594, 0.44238406],
       [0.5493014 , 0.4506986 ],
       [0.56306565, 0.43693438],
       [0.49305546, 0.50694454],
       [0.5080884 , 0.4919116 ],
       [0.5711759 , 0.42882404],
       [0.540348  , 0.45965198],
       [0.5074196 , 0.49258035],
       [0.52853125, 0.4714687 ],
       [0.56322014, 0.4367798 ],
       [0.5405539 , 0.459446  ],
       [0.60848737, 0.39151263]], dtype=float32)

In [12]:
y_validation

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [13]:
validation_indices


['158',
 '460',
 '606',
 '320',
 '1',
 '658',
 '559',
 '92',
 '538',
 '675',
 '419',
 '295',
 '393',
 '409',
 '501',
 '375',
 '286']